In [1]:
import torch
from torch import nn
"""
NiN是 网络中的网络
其想法是在每个像素位置(针对每个高度和宽度)应用一个全连接层。
如果将权重连接到每个空间位置,可以将其视为1*1卷积层,或作为在每个像素位置上独立作用的全连接层。
从另一个角度看，即将空间维度中的每个像素视为单个样本,将通道维度视为不同特征(feature)。

NiN使用由一个卷积层和多个1*1卷积层组成的块。该块可以在卷积神经网络中使用,以允许更多的每像素非线性。
NiN去除了容易造成过拟合的全连接层,将它们替换为全局平均汇聚层（即在所有位置上进行求和）。
该汇聚层通道数量为所需的输出数量(例如,Fashion-MNIST的输出为10)。
移除全连接层可减少过拟合,同时显著减少NiN的参数。
"""
def nin_block(in_channels, out_channels, kernel_size, strides, padding): # NiN块,先是自定义卷积层,之后过两层1*1卷积层
    return nn.Sequential(
    nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
    nn.ReLU(),
    nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),
    nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU())
NiN = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d(output_size=(1, 1)),
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten())